# Practice Methods for PMC and JSON Extractions

Do not use this notebook in your analysis. This is a "scratch pad."

In [1]:
import requests
import json
import pandas as pd

In [2]:
r = requests.get('https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_json/17299597/unicode').json()

In [128]:
def json_extract(obj, key):
    """Recursively fetch values from nested JSON."""
    arr = []

    def extract(obj, arr, key):
        """Recursively search for values of key in JSON tree."""
        if isinstance(obj, dict):
            for k, v in obj.items():
                if isinstance(v, (dict, list)):
                    extract(v, arr, key)
                elif k == key:
                    arr.append(v)
        elif isinstance(obj, list):
            for item in obj:
                extract(item, arr, key)
        return arr

    values = extract(obj, arr, key)
    return values

In [174]:
section_types = json_extract(r, 'section_type')
discuss_indices = []
methods_indices = []
for index, item in enumerate(section_types):
    if item == 'DISCUSS':
        discuss_indices.append(index)
    elif item == 'METHODS':
        methods_indices.append(index)

In [198]:
section_texts = json_extract(r, 'text')
merged_discussion = ''
merged_methods = ''

for index, item in enumerate(section_texts):
    if index in discuss_indices:
        merged_discussion += (item + ' ')
    elif index in methods_indices:
        merged_methods += (item + ' ')


In [199]:
# print(merged_methods)

In [200]:
data_dict = {
    'source': r['source'],
    'date': r['date'],
    'document_id': r['documents'][0]['id'],
    'document_license': r['documents'][0]['infons']['license'],
    'title': r['documents'][0]['passages'][0]['text'],
    'methods': merged_methods,
    'discussion': merged_discussion
}

# print(data_dict)

In [201]:
dct = {k: [v] for k,v in data_dict.items()}
df = pd.DataFrame.from_dict(dct)
df.head()

,source,date,document_id,document_license,title,methods,discussion
0,PMC,20201214,1790863,CC BY,Quantifying Organismal Complexity using a Popu...,Methods Viral Evolution The details of experim...,Discussion To understand how biological comple...
